# 1. BentoML

## 1.1 BentoML

### 1.1.1 Introduction

- FastAPI로 직접 머신러닝 서버 개발
  - 1~2개의 모델을 만들 때는 시도하면서 직접 개발 가능
- 만약 30개~50개의 모델을 만들어야 한다면?
  - 많은 모델을 만들다보니 반복되는 작업이 존재(Config, FastAPI 설정 등)
- 이 조차도 더 빠르게 간단하게 하고 싶다. 더 쉽게 만들고 싶다. 추상화 불가능할까?

- 더 쉬운 개발을 위해 본질적인 “Serving”에 특화된 라이브러리를 원하게 됨
- 이런 목적의 라이브러리들이 점점 등장하기 시작

<br>

### 1.1.2 Serving Infra

- Serving Infra는 모든 회사에서 사용될 수 있는 Base Component

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zYHFbOghZSgg-gq9cNLRgQfK5RrecW65' width=600/>

<br>

- 많은 라이브러리가 등장하고 있음
- 모든 라이브러리는 해결하려고 하는 핵심 문제가 존재
- 어떻게 문제를 해결했는지가 다른 라이브러리

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zYJIl7s2QDP-xmq5epUwpBxdDZ4RKvVP' width=400/>

<br>

### 1.1.3 BentoML이 해결하는 문제

1. Model Serving Infra의 어려움
2. Online Serving의 Monitoring 및 Error Handling
3. Online Serving 퍼포먼스 튜닝의 어려움

<br>

#### 1.1.3.1 Model Serving Infra의 어려움

- Serving을 위해 다양한 라이브러리, Artifact, Asset 등 사이즈가 큰 파일을 패키징
- Cloud Service에 지속적인 배포를 위한 많은 작업이 필요
- BentoML은 CLI로 이 문제의 복잡도를 낮춤(CLI 명령어로 모두 진행 가능하도록)

<br>

#### 1.1.3.2 Online Serving의 Monitoring 및 Error Handling

- Online Serving으로 API 형태로 생성
- Error 처리, Logging을 추가로 구현해야 함
- BentoML은 Python Logging Module을 사용해 Access Log, Prediction Log를 기본으로 제공
- Config를 수정해 Logging도 커스텀할 수 있고, Prometheus 같은 Metric 수집 서버에 전송할 수 있음

<br>

#### 1.1.3.3 Online Serving 퍼포먼스 튜닝의 어려움

- BentoML은 [Adaptive Micro Batch](https://docs.bentoml.org/en/latest/guides/micro_batching.html) 방식을 채택해 동시에 많은 요청이 들어와도 높은 처리량을 보여줌

<br>

### 1.1.4 BentoML 소개

- Serving에 집중하는 가벼운 Library, BentoML
- 2019년부터 개발 시작해서 최근 가파른 성장

- Bento(벤또): 일본의 도시락 요리
- Yatai(야타이): 일본식 포장마차

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1z_i_R9nbxptxIVs1EhY5sCmr2ZlaIISy' width=600/>

<br>

### 1.1.5 BentoML 특징

- 쉬운 사용성
- Online / Offline Serving 지원
- Tensorflow, PyTorch, Keras, XGBoost 등 Major 프레임워크 지원
- Docker, Kubernetes, AWS, Azure 등의 배포 환경 지원 및 가이드 제공
- Flask 대비 100배의 처리량
- 모델 저장소(Yatai) 웹 대시보드 제공
- 데이터 사이언스와 DevOps 사이의 간격을 이어주며 높은 성능의 Serving이 가능하게 함

<br>

## 1.2 BentoML 시작하기

### 1.2.1 BentoML 설치

- BentoML은 python 3.6 이상 버전을 지원
- pyenv 등으로 python version을 3.8으로 설정
- 가상 환경 설정(virutalenv or poetry)

```
pip install bentoml
```

<br>

### 1.2.2 BentoML 사용 Flow

- 모델 학습 코드 생성
- Prediction Service Class 생성
- Prediction Service에 모델 저장 (Pack)
- (Local) Serving
- Docker Image Build (컨테이너화)
- Serving 배포

<br>

#### 1.2.2.1 모델 학습 코드

- Iris Classifier 예제

```python
from sklearn import svm
from sklearn import datasets

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)
```

- (기존에) Serving하기 위해 해야 하는 작업
  - FastAPI 웹 서버 생성
  - Input, Output 정의
  - 의존성 작업 (requirements.txt, Docker 등)

<br>

#### 1.2.2.2 Prediction Service Class 생성

- **BentoService**를 활용해 Prediction Service Class 생성
- 예측할 때 사용하는 API를 위한 Class

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1z_sMxznJZzdP9HfwN1oRHRIaUaUuNDM4' width=800/>

<br>

`@env`

- 파이썬 패키지, install script 등 서비스에 필요한 의존성을 정의

<br>

`@artifacts`

- 서비스에서 사용할 artifact 정의 (sklearn)

<br>

`BentoService`

- `BentoService`를 상속하면, 해당 서비스를 Yatai(모델 이미지 레지스트리)에 저장

<br>

`@api`

- API 생성
- Input과 Output을 원하는 형태(Dataframe, Tensor, JSON 등)으로 선택할 수 있음
- Doc String으로 Swagger에 들어갈 내용을 추가할 수 있음
- `@artifacts`에 사용한 이름을 토대로 `self.artifacts.model`로 접근
  - API에 접근할 때 해당 Method 호출

<br>

#### 1.2.2.3 Prediction Service 저장 (Pack)

학습한 모델을 Prediction Service에 Pack

```python
from sklearn import svm
from sklearn import datasets

clf = svm.SVC(gamma='scale')
iris = datasets.load_iris()
X, y = iris.data, iris.target
clf.fit(X, y)
```

- `bento_service.py`에 정의한 Prediction Service

```python
# bento_service.py에서 정의한 IrisClassifier
from bento_service import IrisClassifier

# IrisClassifier 인스턴스 생성
iris_classifier_service = IrisClassifier()
```

- Model Artifact를 주입
- BentoML Bundle
  - Prediction Service를 실행할 때 필요한 모든 코드, 구성이 포함된 폴더, 모델 제공을 위한 바이너리

```python
# Model Artifact를 Pack
iris_classifier_service.pack('model', clf)

# Model Serving을 위한 서비스를 Disk에 저장
saved_path = iris_classifier_service.save()
```

<br>

- CLI에서 `bento_packer.py` 실행
  - "Saved to ~" 경로가 보임
  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1za6D-bkIxzBOjSj4uXIlFDwTc2dNhWWe' width=1000/>

<br>

- BentoML에 저장된 Prediction Service 확인

```
bentoml list
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zcufMSMgbarKiiIOdaVxiD2pxKF_5iJk' width=800/>

<br>

- BentoML에 저장된 Prediction Service 폴더로 이동 후, 파일 확인
- `tree` 명령어를 통해 파일 구조 확인

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zfHaeEKBnB-qlgb_pGeHIMLqmiBkkX5b' width=500/>

<br>

- 우리가 생성한 `bento_service.py` code와 동일

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zg0naP3r4DTiholXy5GstdI1MsOoKDmx' width=800/>

<br>

- 우리가 생성하지 않은 `__init__.py`

```
create_bento_service_cli
```

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zg2-o5SMMBGxXyeLos9yWpzGdvvwNxxs' width=800/>

<br>

- `bentoml.yml`에 모델의 메타정보가 저장됨
  - 패키지 환경
  - API Input / Output
  - Docs 등

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zhRUdzkUtXUDY2J_lr7Q53R11psJmR9O' width=800/>

<br>

- `Dockerfile`도 자동으로 생성되며, 설정을 가지고 설치하는 코드

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zhV81kdevHwNN60ffw9wo9SFHwkpQFR3' width=800/>

<br>

- 다른 이름의 모델이 Pack 되면 어떻게 되는 지 확인하기 위해 `bento_service.py`의 `IrisClassifier` 클래스를 `IrisClassifier1`로 수정(`bento_packer.py`의 `import` 부분도 수정)한 후 코드 실행

```
python bento_packer.py
```

- `IrisClassifier1`이 새로 생성됨

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zkptJJnLoRjjkIhf4sL_IlCV1DJUL8uq' width=800/>

- `bentoml list`로 확인

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zljga_-if706OhkRQx8Ocldpd6R1ueQT' width=800/>

<br>

#### 1.2.2.4 Serving

- 다음 명령어로 serving

```
bentoml serve IrisClassifier:latest
```

- 웹서버 실행

<br>

- `localhost:5000`로 접근하면 Swagger UI가 보임
- 우리가 생성한 `/predict`을 클릭하면 코드에서 정의한 내용을 볼 수 있고, API Test도 가능

<br>

- BentoML serve한 터미널에선 로그 발생
- 로그는 `~/bentoml/logs`에 저장됨
- `prediction.log`를 확인하면 예측 로그를 확인할 수 있음

<br>

- 터미널에서 `curl`로 Request해도 정상적으로 실행됨

```
curl -i \
  --header "Content-Type: application/json" \
  --request POST \
  --data '[[5.1, 3.5, 1.4, 0.2]]' \
  http://localhost:5000/predict
```

<br>

#### 1.2.2.5 Yatai Service 실행

```
bentoml yatai-service-start
```

- web UI: `localhost:3000`

<br>

#### 1.2.2.6 Docker Image Build

```
bentoml containerize IrisClassifier:latest -t iris-classifier
```

- 조금 기다리면 Container Image가 빌드됨

<br>

- Docker Image로 빌드된 이미지 확인

```
docker images
```

<br>

- docker 명령어나 FastAPI를 사용하지 않고 웹 서버를 띄우고, 이미지 빌드 가능
  - 예전보다 더 적은 리소스로 개발 가능

<br>

## 1.3 BentoML Component

1. BentoService
2. Service Environment
3. Model Artifact
4. Model Artifact Metadata
5. Model Management & Yatai
6. API Function and Adapters
7. Model Serving
8. Labels
9. Retrieving BentoServices
10. WEB UI

<br>

### 1.3.1 BentoService

- `bentoml.BentoService`
  - 예측 서비스를 만들기 위한 베이스 클래스
- `@bentoml.artifacts`
  - 여러 머신러닝 모델 포함할 수 있음
- `@bentoml.api`
  - Input/Output 정의
  - API 함수 코드에서 `self.artifacts.{ARTIFACT_NAME}`으로 접근할 수 있음
- 파이썬 코드와 관련된 종속성 저장

<br>

```python
# bento_svc.py
import bentoml
from bentoml.adapters import JsonInput
from bentoml.frameworks.keras import KerasModelArtifact
from bentoml.service.artifacts.common import PickleArtifact

@bentoml.env(
    pip_packages=['tensorflow', 'scikit-learn', 'pandas'],
    docker_base_image="bentoml/model-server:0.12.1-py38-gpu"
)
@bentoml.artifacts([KerasModelArtifact('model'), PickleArtifact('tokenizer')])
class TensorflowService(bentoml.BentoService):
    @api(input=JsonInput())
    def predict(self, parsed_json):
        return self.artifacts.model.predict(input_data)
```

<br>

```python
# bento_packer.py
from bento_svc import TensorflowService

# OPTIONAL: to remove tf memory limit on our card
config.experimental.set_memory_growth(gpu[0], True)

model = load_model()
tokenizer = load_tokenizer()

bento_svc = TensorflowService()
bento_svc.pack('model', model)
bento_svc.pack('tokenizer', tokenizer)

saved_path = bento_svc.save()
```

<br>

### 1.3.2 Service Environment

- 파이썬 관련 환경, Docker 등을 설정할 수 있음
- `@bentoml.env(infer_pip_packages=True)`
  - import를 기반으로 필요한 라이브러리 추론

<br>

- `requirements_txt_file`을 명시할 수도 있음

```python
@bentoml.env(
    requirements_txt_file="./requirements.txt"
)
class ExamplePredictionService(bentoml.BentoService):
    @bentoml.api(input=DataframeInput(), batch=True)
    def predict(self, df):
        return self.artifacts.model.predict(df)
```

<br>

- `pip_pakcages=[]`를 사용해 버전을 명시할 수 있음

```python
@bentoml.env(
    pip_packages=[
        'scikit-learn==0.24.1',
        'pandas @https://github.com/pypa/pip/archive/1.3.1.zip'
    ]
)
class ExamplePredictionService(bentoml.BentoService):
    @bentoml.api(input=DataframeInput(), batch=True)
    def predict(self, df):
        return self.artifacts.model.predict(df)
```

<br>

- `docker_base_image`를 사용해 Base Image를 지정할 수 있음

<br>

- `setup_sh`를 지정해 Docker Build 과정을 커스텀할 수 있음

```python
@bentoml.env(
    infer_pip_packages=True,
    setup_sh="./my_init_script.sh"
)
class ExamplePredictionService(bentoml.BentoService):
    ...
    
@bentoml.env(
    infer_pip_pakcages=True,
    setup_sh="""\
#!/bin/bash
set -e

apt-get install --no-install-recommends nvidia-driver-430
...
"""
)
class ExamplePredictionService(bentoml.BentoService):
    ...
```

<br>

- `@bentoml.ver`를 사용해 버전 지정할 수 있음

```python
from bentoml import ver, artifacts
from bentoml.service.artifacts.common import PickleArtifact

@ver(major=1, minor=4)
@artifacts([PickleArtifact('model')])
class MyMLService(BentoService):
    pass

svc = MyMLService()
svc.pack("model", trained_classifier)
svc.set_version("2019-08.iteration20")
svc.save()

# The final produced BentoService bundle will have version:
# "1.4.2019-08.iteration20"
```

<br>

### 1.3.3 Model Artifact

- `@bentoml.artifacts`
  - 사용자가 만든 모델을 저장해 pretrain model을 읽어 Serialization, Deserialization
- 여러 모델을 같이 저장할 수 있음
- A 모델의 예측 결과를 B 모델의 Input으로 사용할 경우
- 보통 하나의 서비스 당 하나의 모델을 권장

```python
import bentoml
from bentoml.adapters import DataframeInput
from bentoml.frameworks.sklearn import SklearnModelArtifact
from bentoml.frameworks.xgboost import XgboostModelArtifact

@bentoml.env(infer_pip_packages=True)
@bentoml.artifacts([
    SklearnModelArtifact("model_a"),
    XgboostModelArtifact("model_b")
])
class MyPredictionService(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput(), batch=True)
    def predict(self, df):
        # assume the output of model_a will be the input of model_b in this example
        df = self.artifacts.model_a.predict(df)
        
        return self.artifacts.model_b.predict(df)
```

<br>

```python
svc = MyPredictionService()
svc.pack("model_a", my_sklearn_model_object)
svc.pack("model_b", my_xgboost_model_object)
svc.save()
```

<br>

- 다음 라이브러리의 Artifact를 지원

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1znkBehudZuOGrBFR-adxmD7ddk7FYedq' width=300/>

<br>

### 1.3.4 Model Artifact Metadata

- 해당 모델의 Metadata(Metric - Accuracy, 사용한 데이터셋, 생성한 사람, static 정보 등)
- Pack에서 `metadata` 인자에 넘겨주면 메타데이터 저장
- 메타데이터는 Immutable

```python
svc = MyPredictionService()
svc.pack(
    "model_a",
    my_sklearn_model_object,
    metadata={
        "precision_score": 0.876,
        "created_by": "joe"
    }
)
svc.pack(
    "model_b",
    my_xgboost_model_object,
    metadata={
        "precision_score": 0.792,
        "mean_absolute_error": 0.88
    }
)
svc.save()
```

<br>

Metadata 접근 방법 (1) - CLI

```
bentoml get model:version
```

<br>

Metadata 접근 방법 (2) - REST API

- `bentoml serve` 한 후, `/metadata`로 접근

<br>

Metadata 접근 방법 (3) - Python

```python
from bentoml import load
svc = load("path_to_bento_service")
print(svc.artifacts['model'].metadata)
```

<br>

### 1.3.5 Model Management & Yatai

- BentoService의 `save` 함수는 BentoML Bundle을 `~/bentoml/repository/{서비스 이름}/{서비스 버전}`에 저장

- 모델 리스트 확인

```
bentoml list
```

- 특정 모델 정보 가져오기

```
bentoml get IrisClassifier
```

- YataiService
  - 모델 저장 및 배포를 처리하는 컴포넌트

```
bentoml yatai-service-start
```

<br>

### 1.3.6 API Function and Adapters

- BentoService API는 클라이언트가 예측 서비스에 접근하기 위한 End Point 생성
- Adapter는 Input / Output을 추상화해서 중간 부분을 연결하는 Layer
  - ex) csv 파일 형식으로 예측을 요청할 경우, `DataframeInput`을 사용하고 있으면 내부적으로 `pandas.DataFrame` 객체로 변환하고 API 함수에 전달함

<br>

- `@bentoml.api`를 사용해 입력 받은 데이터를 `InputAdapter` 인스턴스에 넘김
- 데이터 처리하는 함수도 작성할 수 있음

```python
from my_lib import preprocessing, postprocessing, fetch_user_profile_from_database

class ExamplePredictionService(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput(), batch=True)
    def predict(self, df):
        user_profile_column = fetch_user_profile_from_database(df['user_id'])
        df['user_prifile'] = user_profile_column
        model_input = preprocessing(df)
        model_output = self.artifacts.model.predict(model_input)
        return postprocessing(model_output)
```

<br>

- Input 데이터가 이상할 경우 오류 코드를 반환할 수 있음

```python
from typing import List
from bentoml import env, artifacts, api, BentoService
from bentoml.adapters import JsonInput
from bentoml.types import JsonSerializable, InferenceTask # type annotations are optional

@env(infer_pip_packages=True)
@artifacts([SklearnModelArtifact('classifier')])
class MyPredictionService(BentoService):
    
    @api(input=JsonInput(), batch=True)
    def predict_batch(self, parased_json_list: List[JsonSerializable], tasks: List[InferenceTask]):
        model_input = []
        for json, task in zip(parsed_json_list, tasks):
            if "text" in json:
                model_input.append(json["text"])
            else:
                task.discard(http_status=400, err_msg="input json must contain 'text' field")
                
        results = self.artifacts.classifier(model_input)
        
        return results
```

<br>

- 세밀하게 Response를 정의할 수 있음

```python
import bentoml
from bentoml.types import JsonSerializable, InferenceTask, InferenceError, InferenceResult

class MyService(bentoml.BentoService):
    
    @bentoml.api(input=JsonInput(), batch=True)
    def predict(self, parsed_json: JsonSerializable, task: InferenceTask) -> InferenceResult:
        if task.http_headers['Accept'] == "application/json":
            predictions = self.artifact.model.predict([parsed_json])
            return InferenceResult(
                data=predictions[0],
                http_status=200,
                http_headers={"Content-Type": "application/json"}
            )
        else:
            return InferenceError(err_msg="application/json output only", http_status=400)
```

<br>

- `BentoService`가 여러 API를 포함할 수 있음

```python
from my_lib import process_custom_json_format

class ExamplePredictionService(bentoml.BentoService):
    
    @bentoml.api(input=DataframeInput(), batch=True)
    def predict(self, df: pandas.DataFrame):
        return self.artifacts.model.predict(df)
    
    @bentoml.apip(input=JsonInput(), batch=True)
    def predict_json(self, json_arr):
        df = process_custom_json_format(json_arr)
        return self.artifacts.model.predict(df)
```

<br>

### 1.3.7 Model Serving

- `BentoService`가 bento로 저장되면, 여러 방법으로 배포할 수 있음

1. Online Serving
  - 클라이언트가 REST API Endpoint로 근 실시간으로 예측 요청
2. Offline Batch Serving
  - 예측을 계산한 후, Storage에 저장
3. Edge Serving
  - 모바일, IoT Device에 배포

<br>

### 1.3.8 Retrieving BentoServices

- 학습한 모델을 저장한 후, Artifact bundle을 찾을 수 있음
- `--target_dir flag`를 사용

```
bentoml retrieve ModelServe --target_dir=~/bentoml_bundle/
```

<br>

### 1.3.9 WEB UI

- `@bentoml.web_static_content`를 사용하면 웹 프론트엔드에 추가할 수 있음
- 참고 링크: [https://github.com/bentoml/gallery/tree/master/scikit-learn/iris-classifier](https://github.com/bentoml/gallery/tree/master/scikit-learn/iris-classifier)

```python
%%writefile iris_classifier.py
from bentoml import env, artifacts, api, BentoService, web_static_content
from bentoml.adapters import DataframeInput
from bentoml.artifact import SklearnModelArtifact

@env(auto_pip_dependencies=True)
@artifacts([SklearnModelArtifact('model')])
@web_static_content('./static')
class IrisClassifier(BentoService):
    
    @api(input=DataframeInput(), batch=True)
    def test(self, df):
        # Optional pre-processing, post-processing code goes here
        return self.artifacts.model.predict(df)
```

<br>

## 1.4 BentoML으로 Serving 코드 리팩토링하기

- [https://github.com/zzsza/Boostcamp-AI-Tech-Product-Serving/tree/main/part4/01-bentoml/app](https://github.com/zzsza/Boostcamp-AI-Tech-Product-Serving/tree/main/part4/01-bentoml/app)

<br>

### 1.4.1 BentoService 정의하기

- import된 모듈로 패키지 의존성을 추론해 추가
- 마스크 분류 모델에서 사용한 `PytorchModelArtifact` 추가

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1znzlgc5oylJAzsQSFuwRY7Bt8GU5k5yu' width=600/>

<br>

- image input을 사용해서 업로드된 이미지로부터 `imageio.Array`를 함수 인자로 주입
- output은 json으로 클라이언트에게 제공

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zps6baIbVhC0fXVjx7iGAoixVf52bzSo' width=600/>

<br>

- 사용하기 전에 [공식 Document](https://docs.bentoml.org/en/latest/api/adapters.html)를 보는 습관!

<br>

- FastAPI에서 썼던 코드와 거의 동일
- `@api` 데코레이터가 붙은 Method 말고도 `self.transform` 같은 추가 Method도 사용 가능

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zqicCG6Dnp1JBtBkKXH04OMH400x5S1i' width=600/>

<br>

### 1.4.2 Model Pack

- Service를 초기화하고 Model Load

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zsLP6sI9XzxCZBZsh4OGH72wRVBbsCFT' width=600/>

<br>

- 서비스를 Pack하고 Yatai에 저장

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zsdcJeB5WS9-NlNP4NXQxezZtcZSgQLp' width=600/>

<br>

- `http://localhost:%port`로 접속하면 Swagger를 확인할 수 있음
- Test Execute하면 정상적으로 값이 Return!

<br>

- 기존 Requests하는 URL을 5001로 변경하면 됨